# Step 1: Data

### Import libraries

In [79]:
import library #library of functions
import pandas as pd
import matplotlib.pyplot as plt
import datetime 
import numpy as np
import os
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
#nltk.download('punkt')
#nltk.download('stopwords')

We only need some columns values of the Airbnb_Texas_Rentals.csv so we filter the result by:

In [65]:
# List of column values needed
cols = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description', 'latitude', 'longitude', 'title', 'url']
# Read the csv file
airbnbrentals = pd.read_csv('Airbnb_Texas_Rentals.csv', sep=',', encoding='utf-8', usecols=cols)

In [66]:
airbnbrentals.loc[0:2,]

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
2,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...


# Step 2: Create documents

In this step we are creating i tsv files to save all the information. 

In [72]:
# Create directory to save the documents if doesn't exist
directory = "documents"
if not os.path.exists(directory):
    os.makedirs(directory)

# Create all tsv files, each per row
nRowsOrFiles = 100 #airbnbrentals.shape[0] #this variable is the number of rows or files
for i in range(0, nRows): 
    #Write the tsv file
    airbnbrentals.loc[i:i,].to_csv('documents/doc_'+str(i)+'.tsv', sep='\t', encoding='utf-8', index = False)

In [74]:
# Read the tsv file
pd.read_csv('documents/doc_1.tsv', sep='\t', encoding='utf-8')

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...


# Step 3: Search Engine

As a first common step, we have to preprocess the documents by:

- Removing stopwords
- Removing punctuation
- Stemming
- Anything else you think it's needed

In [82]:
# This function takes all documents and removes punctuation, stop words and do stemming
modifyDocs()

In [85]:
#Example of a tsv document cleaned
pd.read_csv('documents/doc_99.tsv', sep='\t', encoding='utf-8')

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,$75,1,Colleyville,November 2015,love guest suit with all the amen of home quee...,32.890392,-97.181868,charm suburban escap,https://www.airbnb.com/rooms/17873586?location...


## 3.1) Conjunctive query

At this moment, we are taking into account the description and title of each document. It means that the first Search Engine will evaluate queries with respect to the aforementioned information.

### 3.1.1) Create your index!